# Introduction to NLP Fundamentals in Tensorflow

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
2.14.0-dev20230730


In [2]:
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")

# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

[INFO] 'helper_functions.py' already exists, skipping download.


We will be using RNN to train textual information

**Basic Structure of RNN**
* Input layer
* text_vectorizer(input)
* embedding(x)
* LSTM -> activation = tanh
* output_Dense -> activation = sigmoid

### Get the text dataset

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
    unzip_data("nlp_getting_started.zip")
else:
  print("[INFO] already exists, skipping download.")

[INFO] already exists, skipping download.


### Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
#Shuffle to train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [7]:
# test_data
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
#How many examples in each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# visualise some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
print(random_index)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  index,text,target = row
  print(f"Target: {target}, {'(real disaster)' if target > 0 else '(not a real disaster)'}")
  print(f"Text:\n{text}\n")
  print("---")

677
Target: 0, (not a real disaster)
Text:
One Direction Is my pick for http://t.co/y9WvqKGbBI Fan Army #Directioners http://t.co/S5F9FcOmp8

---
Target: 0, (not a real disaster)
Text:
http://t.co/DeQQOpSP4f: Iger's 3 words that wrecked Disney's stock http://t.co/LbKvFqRpgT http://t.co/3rVa5uvt0P

---
Target: 0, (not a real disaster)
Text:
Jay and alexis broke up there goes all your faith and goals people... evacuate yourself

---
Target: 0, (not a real disaster)
Text:
New illustration for the L.A. Times: http://t.co/qYn6KxJSTl #illustration #subway

---
Target: 0, (not a real disaster)
Text:
The Drowning Girl by Caitlin R. Kiernan Centipede Press Signed numbered Limited - Full reÛ_ http://t.co/tCJfCkXdZL http://t.co/EmTXtGO4CE

---


In [10]:
len(train_df_shuffled)

7613

In [11]:
#Splitting the data to create validation -> (train + validation)
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
#Check length
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check few sample
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

1. tokenization -> can get too big as the number of text sample increases
2. Embedding -> richer representation of relationship be between tokens, deeper the realtion between words, more the number/embedding for that token

In [14]:
import tensorflow as tf

# Use the default TextVectorization variables
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                                    standardize="lower_and_strip_punctuation", # how to process text
                                                    split="whitespace", # how to split tokens
                                                    ngrams=None, # create groups of n-words?
                                                    output_mode="int", # how to map tokens to numbers
                                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode="int",
                                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@Joelsherman1 DW was on his way to a better career than Chipper and that wasn't derailed until 2014. Shame.      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    1,   23,   11,   79,  147,    5,    3,  441, 1221,   76,
           1,    7,   16, 1051]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [21]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/mNsy1QR7bq

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0333038 , -0.04640305, -0.00079908, ...,  0.03080088,
          0.02150322, -0.01348457],
        [-0.04250634, -0.00135197,  0.04288374, ..., -0.01781004,
          0.01362566,  0.01512757],
        [ 0.00465859,  0.03839483,  0.02802278, ..., -0.04299737,
          0.02527736,  0.0056807 ],
        ...,
        [ 0.02868957, -0.00700942,  0.0331139 , ..., -0.01618806,
          0.01504565,  0.01664117],
        [ 0.0405761 , -0.03174063, -0.04751214, ..., -0.02444764,
          0.04344641,  0.02165746],
        [-0.02290688, -0.00010388, -0.03192502, ..., -0.02150967,
          0.01739729,  0.04936871]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.0333038 , -0.04640305, -0.00079908,  0.04687523,  0.00969108,
        0.04183871, -0.02434431,  0.01112809,  0.01454909,  0.00355725,
        0.01508242,  0.03205324, -0.0139525 ,  0.00695089,  0.04764632,
        0.04594972, -0.02236244,  0.03787446,  0.0355726 , -0.02461576,
        0.01467428, -0.02395367,  0.00778165,  0.03878135, -0.0479964 ,
        0.03400121, -0.02646736, -0.01169569,  0.01297725, -0.01847421,
       -0.04177887, -0.04357892, -0.03009887,  0.01751209,  0.03341943,
        0.04008106,  0.03873212, -0.04817969,  0.02188667,  0.04695858,
       -0.01202103, -0.01340287,  0.02675045, -0.00389322,  0.00628815,
       -0.01847651,  0.03741478, -0.0225047 , -0.019374  ,  0.00787682,
        0.00507209, -0.0061603 , -0.03618749, -0.03395033, -0.02141045,
       -0.04931509,  0.04367323,  0.04697097,  0.02725736,  0.02678101,
        0.04695481,  0.02637411,  0.04491968,  0.03791643, -0.00432162,
       -0.007993

### Modelling a text dataset
More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data



In [24]:
#model 0
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
from helper_functions import calculate_results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 1: A simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230815-002822


Epoch 1/5
215/215 [==============================] - 5s 18ms/step - loss: 0.6098 - accuracy: 0.6917 - val_loss: 0.5361 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4413 - accuracy: 0.8196 - val_loss: 0.4688 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3462 - accuracy: 0.8624 - val_loss: 0.4586 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2846 - accuracy: 0.8920 - val_loss: 0.4637 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2377 - accuracy: 0.9121 - val_loss: 0.4763 - val_accuracy: 0.7861


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4763 - accuracy: 0.7861


[0.4762728214263916, 0.7860892415046692]

#### Recurrent Neural Network
* RNN's are useful for sequence data
* The premise of a recurrent neural network is to use the representation of previous input to aid the representation of a later input

### Model 2 : LSTM -> Long short term memory

Input(text) -> Toeknize -> Embedding -> Layers(RNN/Dense) -> Output(label probability)

In [31]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

model_2_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)

print(x.shape)

x = tf.keras.layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x) # return vector for whole sequence

print(x.shape)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

(None, 15, 128)


(None, 64)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                  

In [32]:
# Fit model
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230815-002830
Epoch 1/5


215/215 [==============================] - 7s 23ms/step - loss: 0.5086 - accuracy: 0.7489 - val_loss: 0.4605 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3261 - accuracy: 0.8710 - val_loss: 0.5178 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2293 - accuracy: 0.9139 - val_loss: 0.5532 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1710 - accuracy: 0.9404 - val_loss: 0.5950 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1197 - accuracy: 0.9555 - val_loss: 0.8668 - val_accuracy: 0.7612


In [33]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.01166967],
       [0.57826775],
       [0.9989392 ],
       [0.04926588],
       [0.00222008],
       [0.9993309 ],
       [0.6216256 ],
       [0.9986928 ],
       [0.9988696 ],
       [0.03266491]], dtype=float32)

In [34]:
# Convert preds to laels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [36]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 76.11548556430446,
 'precision': 0.763026475717381,
 'recall': 0.7611548556430446,
 'f1': 0.758712452353006}

#### Model 3: GRU

1. Gated recurrent unit is effective and popular
2. GRU cell has similar features to LSTM cell but has less paramenter

In [37]:
# Build GRU RNN
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.GRU(64, return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs,outputs)

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.2315 - accuracy: 0.9059 - val_loss: 0.5313 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1598 - accuracy: 0.9419 - val_loss: 0.5950 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1282 - accuracy: 0.9534 - val_loss: 0.7472 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1096 - accuracy: 0.9604 - val_loss: 0.7563 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0921 - accuracy: 0.9658 - val_loss: 0.7266 - val_accuracy: 0.7861


In [38]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[9.1570362e-02],
       [8.2884812e-01],
       [9.9961436e-01],
       [4.2820368e-02],
       [9.7794749e-04],
       [9.8932719e-01],
       [6.5318811e-01],
       [9.9975985e-01],
       [9.9948162e-01],
       [6.2755126e-01]], dtype=float32)

In [39]:
# Convert model 3 pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [40]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 78.60892388451444,
 'precision': 0.7885525005575463,
 'recall': 0.7860892388451444,
 'f1': 0.7839649054352522}

#### Model4 - Bidirectional RNN

In [41]:
# Trying out on my own
text_vectorizer_exp = tf.keras.layers.TextVectorization(max_tokens=10000,
                                      output_sequence_length=15)
text_vectorizer_exp.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer_exp(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4_exp = tf.keras.Model(inputs,outputs)
model_4_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_4_history_exp = model_4_exp.fit(train_sentences,
                                      train_labels,
                                      validation_data=(val_sentences,val_labels),
                                      epochs=5)

Epoch 1/5
215/215 [==============================] - 8s 23ms/step - loss: 0.5057 - accuracy: 0.7475 - val_loss: 0.4529 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3100 - accuracy: 0.8752 - val_loss: 0.5229 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2032 - accuracy: 0.9218 - val_loss: 0.5696 - val_accuracy: 0.7507
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1393 - accuracy: 0.9550 - val_loss: 0.6795 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1019 - accuracy: 0.9669 - val_loss: 0.7023 - val_accuracy: 0.7651


Normal RNN's go from left to right

Bi-directional go from right to left as well as left to right

In [42]:
# model 4
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    output_sequence_length=15)
text_vectorizer.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4 = tf.keras.Model(inputs,outputs)

model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4_history = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 6s 21ms/step - loss: 0.5112 - accuracy: 0.7438 - val_loss: 0.4576 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3151 - accuracy: 0.8720 - val_loss: 0.5067 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2158 - accuracy: 0.9178 - val_loss: 0.5578 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1535 - accuracy: 0.9470 - val_loss: 0.6357 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1112 - accuracy: 0.9637 - val_loss: 0.6426 - val_accuracy: 0.7651


In [43]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.06488725],
       [0.84897393],
       [0.9985574 ],
       [0.11038362],
       [0.00617259],
       [0.993626  ],
       [0.90116763],
       [0.9991002 ],
       [0.9988882 ],
       [0.23154856]], dtype=float32)

In [44]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 76.50918635170603,
 'precision': 0.7650786674444272,
 'recall': 0.7650918635170604,
 'f1': 0.7639596899608797}

#### 1D CNN

In [46]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = tf.keras.layers.Conv1D(32,
                                 5,
                                 activation="relu",
                                 padding="valid")
conv_1d_output = conv_1d(embedding_test)

max_pool = tf.keras.layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [47]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01017285,  0.07277857, -0.02530532, ..., -0.03078235,
          0.01014482,  0.05285911],
        [ 0.05055342,  0.04676313, -0.01137518, ...,  0.00098078,
          0.01893992,  0.02824483],
        [-0.02070136,  0.03087352,  0.02875127, ...,  0.00207526,
         -0.03519569,  0.00745272],
        ...,
        [ 0.03888407,  0.02960712, -0.00446845, ...,  0.00203666,
          0.01789256,  0.00611549],
        [ 0.03888407,  0.02960712, -0.00446845, ...,  0.00203666,
          0.01789256,  0.00611549],
        [ 0.03888407,  0.02960712, -0.00446845, ...,  0.00203666,
          0.01789256,  0.00611549]]], dtype=float32)>

In [48]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.        , 0.01767842, 0.0385819 ,
         0.        , 0.02131273, 0.01450205, 0.        , 0.        ,
         0.        , 0.0134118 , 0.        , 0.        , 0.02958664,
         0.        , 0.        , 0.        , 0.04214575, 0.        ,
         0.01086291, 0.        , 0.00996938, 0.        , 0.00146396,
         0.08355489, 0.12155888, 0.        , 0.        , 0.        ,
         0.        , 0.00338308],
        [0.01766491, 0.04526496, 0.02126707, 0.        , 0.        ,
         0.        , 0.01291308, 0.04069066, 0.        , 0.02616296,
         0.        , 0.01298308, 0.        , 0.0661578 , 0.        ,
         0.        , 0.00914833, 0.00108195, 0.02483499, 0.03203433,
         0.        , 0.04512188, 0.0059    , 0.00035662, 0.        ,
         0.03618221, 0.07289198, 0.02896196, 0.        , 0.00214834,
         0.        , 0.02138388],
        [0.07935195, 0.04512736, 0.        , 0.    

In [49]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.07935195, 0.04526496, 0.02126707, 0.04577719, 0.0385819 ,
        0.01073653, 0.02220355, 0.04069066, 0.00272196, 0.02616296,
        0.        , 0.05819859, 0.01649983, 0.0661578 , 0.08215996,
        0.02110979, 0.02091281, 0.00108195, 0.05034259, 0.03203433,
        0.0571812 , 0.04512188, 0.11497744, 0.00035662, 0.0114972 ,
        0.08355489, 0.12155888, 0.02896196, 0.00999351, 0.06117973,
        0.05301882, 0.06646241]], dtype=float32)>

In [55]:
# Create 1D CNN layer to model sequences
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5,
                           padding="valid",
                           activation="relu")(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_5 = tf.keras.Model(inputs,outputs)

model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5_history = model_5.fit(train_sentences,train_labels,
                              validation_data=(val_sentences,val_labels),
                              epochs=5)

Epoch 1/5


215/215 [==============================] - 5s 18ms/step - loss: 0.1629 - accuracy: 0.9400 - val_loss: 0.7362 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1041 - accuracy: 0.9616 - val_loss: 0.8716 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0819 - accuracy: 0.9680 - val_loss: 0.9449 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.0705 - accuracy: 0.9720 - val_loss: 1.0329 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.0610 - accuracy: 0.9750 - val_loss: 1.1175 - val_accuracy: 0.7756


In [56]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 913us/step


array([[2.5231427e-01],
       [6.1509359e-01],
       [9.9994767e-01],
       [5.4715343e-02],
       [3.1011024e-07],
       [9.9792457e-01],
       [9.9197024e-01],
       [9.9984097e-01],
       [9.9999833e-01],
       [6.6484582e-01]], dtype=float32)

In [58]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [59]:
model_5_results = calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 77.55905511811024,
 'precision': 0.7765218346613636,
 'recall': 0.7755905511811023,
 'f1': 0.7739772681043204}

In [60]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 6 - Tensorflow Hub pretrained models
USE-extractor - universal sentence encoder (USE)

In [65]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,
                      "When you can the unvidersal sentence encider on a sentence, it turns into into numbers"])
print(embed_sample[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [66]:
embed_sample

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157027,  0.0248591 ,  0.02878048, ..., -0.00186125,
         0.02315824, -0.01485021],
       [ 0.06090041, -0.08857404,  0.01838204, ..., -0.006537  ,
         0.03106262,  0.0370101 ]], dtype=float32)>

Use the above USE"" embedding layer for a normal type of model

In [69]:
# Create a keras layer using the USE"" pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer(hub.load("https://tfhub.dev/google/universal-sentence-encoder/4"),
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [70]:
sentence_encoder_layer

In [73]:
# Create  sequential layer
model_6 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(1,activation="sigmoid")
])

model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6_history = model_6.fit(train_sentences,train_labels,
                               validation_data=(val_sentences,val_labels),
                               epochs=5)

Epoch 1/5
215/215 [==============================] - 3s 7ms/step - loss: 0.6482 - accuracy: 0.7409 - val_loss: 0.6123 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5808 - accuracy: 0.7932 - val_loss: 0.5627 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5377 - accuracy: 0.7989 - val_loss: 0.5309 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5089 - accuracy: 0.7986 - val_loss: 0.5098 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4888 - accuracy: 0.8011 - val_loss: 0.4954 - val_accuracy: 0.7900


In [74]:
model_6_pred_probs=model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.3684933 ],
       [0.6704134 ],
       [0.85510314],
       [0.3477083 ],
       [0.63400686],
       [0.72157234],
       [0.83046424],
       [0.83513236],
       [0.7626972 ],
       [0.19402333]], dtype=float32)

In [75]:
model_6_preds=tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [76]:
model_6_results = calculate_results(val_labels,model_6_preds)
model_6_results

{'accuracy': 79.00262467191601,
 'precision': 0.7907348439526268,
 'recall': 0.7900262467191601,
 'f1': 0.7887751851796453}

In [77]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
# Challenge, beat the baseline
